In [49]:
with open("Grammar.txt", 'r', encoding='utf-8') as f:
    rows = f.readlines()

state_table = {}
new_rows = []
states = []

for i in range(len(rows)):
    splited = rows[i].split()
    right = splited[2:]
    
    state_table[i + 1] = {}
    new_rows.append([str(i + 1), splited[0], splited[1]])
    for j in range(len(right)):
        state = str((i + 1, j)).replace(' ', '')
        states.append(state)
        new_rows[i] += [state, right[j]]
    
    new_rows[i] += ['(' + str(i + 1) + ',' + str(len(right)) + ')']
    # print(new_rows[i])
    new_rows[i][-1] += '\n'
    new_rows[i] = ' '.join(new_rows[i])

for i in range(1, len(new_rows)):
    new_rows[i] = new_rows[i].lstrip()
# new_rows
rows = ''.join(new_rows)

# rows
with open('newGrammar.txt', 'w', encoding='utf-8') as f:
    f.writelines(rows)

In [50]:
with open("GrammarExample.txt", 'r', encoding='utf-8') as f:
    rows = f.readlines()

# with open('newGrammar.txt', 'r', encoding='utf-8') as f:
#     rows = f.readlines()

states2 = []
for row in rows:
    splited = row.split(' ')[3:]
    for i in range(0, len(splited), 2):
        states2.append(splited[i].replace('\n', ''))

states2

['(1,0)',
 '(1,1)',
 '(1,2)',
 '(2,0)',
 '(2,1)',
 '(2,2)',
 '(2,3)',
 '(3,0)',
 '(3,1)',
 '(4,0)',
 '(4,1)']

In [51]:
rows[1].split(' ')[3:][0::2]

['(2,0)', '(2,1)', '(2,2)', '(2,3)\n']

In [52]:
rows[1].split(' ')[3:][1::2]

['IDList', ',', 'ID']

In [53]:
def str_to_tuple(s):
    lst = s[1:-1].split(',')
    return (int(lst[0]), int(lst[1]))

In [54]:
state_symbol_dict = {}
for row in rows:
    row_states = row.split(' ')[3:][0::2]
    row_symbols = row.split(' ')[3:][1::2]
    for i in range(len(row_states) - 1):
        st, sym = row_states[i].replace('\n', ''), row_symbols[i].replace('\n', '')
        st = str_to_tuple(st)
        state_symbol_dict[st] = sym
    st = row_states[len(row_states) - 1].replace('\n', '')
    st = str_to_tuple(st)
    state_symbol_dict[st] = 'empty'

In [55]:
state_symbol_dict

{(1, 0): 'int',
 (1, 1): 'IDList',
 (1, 2): 'empty',
 (2, 0): 'IDList',
 (2, 1): ',',
 (2, 2): 'ID',
 (2, 3): 'empty',
 (3, 0): 'ID',
 (3, 1): 'empty',
 (4, 0): 'id',
 (4, 1): 'empty'}

In [56]:
rows[1].split(' ')[3:]

['(2,0)', 'IDList', '(2,1)', ',', '(2,2)', 'ID', '(2,3)\n']

In [57]:
for i in range(len(states2)):
    states2[i] = str_to_tuple(states2[i])

In [58]:
state_used = []
# Надо построить БАЗУ и ЗАМЫКАНИЕ

terminals = ['int', ',', 'id']
base_closure = []
closures = []

def spusk(base, closure):
    for current_state in base:
        rule_number, state_number = current_state
        if current_state not in closure:
            closure.append(current_state)

        state_used.append(current_state)
        cur_states, cur_symbols = rows[rule_number - 1].split(' ')[3:][0::2], rows[rule_number - 1].split(' ')[3:][1::2]
        
        if state_number != len(cur_states) -1:
            symbol = cur_symbols[state_number]
            if symbol in terminals:
                return closure
            else:
                for row in rows:
                    splited2 = row.split(' ')
                    left_symbol = splited2[1]
                    state = str_to_tuple(splited2[3])
                    if symbol == left_symbol and state != current_state:
                        if state in states2:
                            states2.remove(state)
                        spusk([state], closure)

    return closure

while len(states2) != 0:
    state = states2[0]
    states2.pop(0)
    base = [state]
    if state[1] != 0:
        prev_state = (state[0], state[1] - 1)
        for cl in closures:
            if prev_state in cl:
                for cl_state in cl:
                    next_state = (cl_state[0], cl_state[1] + 1)
                    if state_symbol_dict[cl_state] == state_symbol_dict[prev_state] and cl_state != prev_state and next_state in states2:
                        base.append(next_state)
                        states2.remove(next_state)

    # print("BASE: ", base)
    if state not in state_used:
        closure = spusk(base, [])
        closures.append(closure.copy())
        base_closure.append({"base": base, "closure": closure.copy()})

base_closure

[{'base': [(1, 0)], 'closure': [(1, 0)]},
 {'base': [(1, 1)], 'closure': [(1, 1), (2, 0), (3, 0), (4, 0)]},
 {'base': [(1, 2), (2, 1)], 'closure': [(1, 2), (2, 1)]},
 {'base': [(2, 2)], 'closure': [(2, 2), (4, 0)]},
 {'base': [(2, 3)], 'closure': [(2, 3)]},
 {'base': [(3, 1)], 'closure': [(3, 1)]},
 {'base': [(4, 1)], 'closure': [(4, 1)]}]

In [59]:
state_num = {}
for i, d in enumerate(base_closure):
    base, closure = d['base'], d['closure']
    state_num[i + 1] = closure

In [60]:
state_num

{1: [(1, 0)],
 2: [(1, 1), (2, 0), (3, 0), (4, 0)],
 3: [(1, 2), (2, 1)],
 4: [(2, 2), (4, 0)],
 5: [(2, 3)],
 6: [(3, 1)],
 7: [(4, 1)]}

In [61]:
state_num_reversed = {}
for st in state_num:
    for val in state_num[st]:
        state_num_reversed[val] = st

In [62]:
state_num_reversed

{(1, 0): 1,
 (1, 1): 2,
 (2, 0): 2,
 (3, 0): 2,
 (4, 0): 4,
 (1, 2): 3,
 (2, 1): 3,
 (2, 2): 4,
 (2, 3): 5,
 (3, 1): 6,
 (4, 1): 7}

In [63]:
all_symbols = list(set(list(state_symbol_dict.values())))

In [64]:
state_table = {}
R_states = []
R_rules = []
for i in state_num:
    state_table[i] = {}
    closure = state_num[i]
    for cl_state in closure:
        num_rule, num_in_row = cl_state
        row = rows[num_rule - 1].split(' ')[3:]
        states, symbols = row[0::2], row[1::2]
        if num_in_row != len(symbols):
            symbol = symbols[num_in_row]
            next_state = str_to_tuple(states[num_in_row + 1].replace('\n', ''))
            state_table[i][symbol] = f"S{state_num_reversed[next_state]}"
        else:
            R_states.append(i)
            R_rules.append(num_rule)

state_table

{1: {'int': 'S2'},
 2: {'IDList': 'S3', 'ID': 'S6', 'id': 'S7'},
 3: {',': 'S4'},
 4: {'ID': 'S5', 'id': 'S7'},
 5: {},
 6: {},
 7: {}}

In [65]:
state_num_reversed

{(1, 0): 1,
 (1, 1): 2,
 (2, 0): 2,
 (3, 0): 2,
 (4, 0): 4,
 (1, 2): 3,
 (2, 1): 3,
 (2, 2): 4,
 (2, 3): 5,
 (3, 1): 6,
 (4, 1): 7}

In [66]:
R_states

[3, 5, 6, 7]

In [67]:
state_table

{1: {'int': 'S2'},
 2: {'IDList': 'S3', 'ID': 'S6', 'id': 'S7'},
 3: {',': 'S4'},
 4: {'ID': 'S5', 'id': 'S7'},
 5: {},
 6: {},
 7: {}}

In [68]:
state_table

{1: {'int': 'S2'},
 2: {'IDList': 'S3', 'ID': 'S6', 'id': 'S7'},
 3: {',': 'S4'},
 4: {'ID': 'S5', 'id': 'S7'},
 5: {},
 6: {},
 7: {}}

In [69]:
state_table

{1: {'int': 'S2'},
 2: {'IDList': 'S3', 'ID': 'S6', 'id': 'S7'},
 3: {',': 'S4'},
 4: {'ID': 'S5', 'id': 'S7'},
 5: {},
 6: {},
 7: {}}

In [70]:
def spusk_r(used_symbols, left_symbol, num_rule, st):
    if left_symbol == "S":
        state_table[st]['empty'] = f'R{num_rule}'
        used_symbols.append(left_symbol)

    if left_symbol in used_symbols:
        return

    for row in rows:
        splited = row.split(' ')
        right = splited[3:]
        right_states = right[0::2]
        right_symbols = right[1::2]

        for i, new_sym in enumerate(right_symbols):
            if new_sym == left_symbol:
                next_state = str_to_tuple(right_states[i + 1].replace('\n', ''))
                if next_state[1] == len(right_states) - 1:
                    # used_symbols.append(left_symbol)
                    state_table[st]['empty'] = f'R{num_rule}'
                    new_left_symbol = splited[1]
                    if new_left_symbol != left_symbol:
                        used_symbols.append(new_left_symbol)
                        spusk_r(used_symbols, new_left_symbol, num_rule, st)
                else:
                    next_symbol = right_symbols[i + 1]
                    if next_symbol in terminals:
                        used_symbols.append(next_symbol)
                        state_table[st][next_symbol] = f'R{num_rule}'
                    else:
                        spusk_r(used_symbols, next_symbol, num_rule, st)
    return

In [71]:
len(R_states)

4

In [72]:
for i, d in state_table.items():
    # print(i, d)
    # print("i: ", i)
    used_symbols = []
    if i in R_states:
        R_states.pop(0)
        num_rule = R_rules[0]
        R_rules.pop(0)

        left_symbol = rows[num_rule - 1].split(' ')[1]
        print(state_table[i])
        print(left_symbol, num_rule, i)
        spusk_r(used_symbols, left_symbol, num_rule, i)
        print(state_table[i])
    print()



{',': 'S4'}
S 1 3
{',': 'S4', 'empty': 'R1'}


{}
IDList 2 5
{'empty': 'R2', ',': 'R2'}

{}
IDList 3 6
{'empty': 'R3', ',': 'R3'}

{}
ID 4 7
{'empty': 'R4'}



In [79]:
state_table[7][','] = 'R4'

In [80]:
import pickle

with open("DKA2.pkl", "wb") as f:
    pickle.dump(state_table, f)

# with open("DKA.pkl", "wb") as f:
#     pickle.dump(state_table, f)

In [75]:
rule_symbol = {}
for row in rows:
    # print(row)
    splited = row.split(' ')
    num_rule, symbol = int(splited[0]), splited[1]
    rule_symbol[num_rule] = symbol

In [116]:
a = [0, 1, 2]
n = 2
a = a[:len(a)-n + 1]
a

[0, 1]

In [88]:
rule_lengths = {}
for i, row in enumerate(rows):
    symbols = row.split(' ')[3:][1::2]
    rule_lengths[i + 1] = len(symbols)

with open("rule_lengths", "wb") as f:
    pickle.dump(rule_lengths, f)

In [76]:
# terminals = ['NUM', 'id', '(', ')', 'true', 'false', 'int', 'void', 'bool', '+', '-', '*', '/', '||', '&&', '<', '>', '=', '<=', '>=', '!=', '==', 'if', 'while', '{', '}', ',', 'else', 'return']
with open("terminals2.pkl", "wb") as f:
    pickle.dump(terminals, f)

with open("rule_symbol2.pkl", "wb") as f:
    pickle.dump(rule_symbol, f)

In [77]:
# terminals = ['NUM', 'id', '(', ')', 'true', 'false', 'int', 'void', 'bool', '+', '-', '*', '/', '||', '&&', '<', '>', '=', '<=', '>=', '!=', '==', 'if', 'while', '{', '}', ',', 'else', 'return']
# with open("terminals.pkl", "wb") as f:
#     pickle.dump(terminals, f)

# with open("rule_symbol.pkl", "wb") as f:
#     pickle.dump(rule_symbol, f)

In [78]:
state_table

{1: {'int': 'S2'},
 2: {'IDList': 'S3', 'ID': 'S6', 'id': 'S7'},
 3: {',': 'S4', 'empty': 'R1'},
 4: {'ID': 'S5', 'id': 'S7'},
 5: {'empty': 'R2', ',': 'R2'},
 6: {'empty': 'R3', ',': 'R3'},
 7: {'empty': 'R4'}}